## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install roboflow pillow matplotlib seaborn tqdm numpy

In [ ]:
# Clone the git repository with the code
import os
if not os.path.exists('Dice-Detection'):
    !git clone https://github.com/Adr44mo/Dice-Detection.git
    %cd Dice-Detection
else:
    %cd Dice-Detection

# Add src to path
import sys
sys.path.append('./src')

## 2. Import Libraries

In [ ]:
import torch
import torch.utils.data as data
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
import os

# Import custom modules
from src.dataset import DiceDetectionDataset, collate_fn
from src.model import get_fasterrcnn_model, save_model_checkpoint
from src.training import train_one_epoch, evaluate, get_optimizer, get_lr_scheduler
from src.metrics import evaluate_map, print_metrics
from src.augmentation import ClassAwareSampler, MosaicAugmentation, apply_random_augmentations
from src.visualization import (
    plot_class_distribution,
    plot_training_history,
    display_sample_batch,
    visualize_predictions,
    plot_ap_comparison
)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 3. Download Dataset from Roboflow

In [ ]:
from roboflow import Roboflow

# TODO: Add your Roboflow API key
rf = Roboflow(api_key="YOUR_API_KEY")

# Download the dice dataset
project = rf.workspace("workspace-spezm").project("dice-0sexk")
dataset = project.version(1).download("coco")

print(f"Dataset downloaded to: {dataset.location}")

## 4. Prepare Datasets

In [ ]:
# Set dataset paths
DATASET_PATH = dataset.location
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
VAL_PATH = os.path.join(DATASET_PATH, "valid")
TEST_PATH = os.path.join(DATASET_PATH, "test")

# Check if test set exists
has_test_set = os.path.exists(TEST_PATH) and os.path.exists(os.path.join(TEST_PATH, "_annotations.coco.json"))
print(f"Test set available: {has_test_set}")

# Create datasets
train_dataset = DiceDetectionDataset(
    root_dir=TRAIN_PATH,
    annotation_file="_annotations.coco.json",
    split="train"
)

val_dataset = DiceDetectionDataset(
    root_dir=VAL_PATH,
    annotation_file="_annotations.coco.json",
    split="val"
)

# Create test dataset if available
if has_test_set:
    test_dataset = DiceDetectionDataset(
        root_dir=TEST_PATH,
        annotation_file="_annotations.coco.json",
        split="test"
    )
    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")
else:
    test_dataset = None
    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    print(f"No test set - will use validation set for final evaluation")

print(f"Number of classes: {train_dataset.num_classes}")

In [ ]:
# Visualize class distribution
train_distribution = train_dataset.get_class_distribution()
print("Training set class distribution:")
for class_name, count in sorted(train_distribution.items()):
    print(f"  {class_name}: {count}")

plot_class_distribution(train_distribution)

## 5. Setup Class-Aware Sampling

Class-aware sampling ensures that each class appears roughly equally during training, regardless of the original class imbalance.

In [ ]:
# Hyperparameters
BATCH_SIZE = 4
NUM_WORKERS = 2
NUM_EPOCHS = 10
LEARNING_RATE = 0.005
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Augmentation settings
USE_CLASS_AWARE_SAMPLING = True
USE_MOSAIC_AUGMENTATION = True
MOSAIC_PROB = 0.5

print(f"Training on: {DEVICE}")
print(f"Class-aware sampling: {USE_CLASS_AWARE_SAMPLING}")
print(f"Mosaic augmentation: {USE_MOSAIC_AUGMENTATION} (prob={MOSAIC_PROB})")

In [ ]:
# Create class-aware sampler
if USE_CLASS_AWARE_SAMPLING:
    train_sampler = ClassAwareSampler(
        train_dataset,
        samples_per_epoch=len(train_dataset) * 2,  # Sample more to see each class
        balance_by='dice_value'
    )
    shuffle = False  # Don't shuffle when using custom sampler
    print(f"Using class-aware sampler with {len(train_sampler)} samples per epoch")
else:
    train_sampler = None
    shuffle = True
    print("Using standard random sampling")

## 6. Wrapper Dataset for Mosaic Augmentation

In [ ]:
class AugmentedDataset(torch.utils.data.Dataset):
    """Wrapper dataset that applies mosaic augmentation"""
    
    def __init__(self, base_dataset, use_mosaic=True, mosaic_prob=0.5):
        self.base_dataset = base_dataset
        self.use_mosaic = use_mosaic
        
        if use_mosaic:
            self.mosaic = MosaicAugmentation(
                base_dataset,
                output_size=(640, 640),
                prob=mosaic_prob
            )
    
    def __len__(self):
        return len(self.base_dataset)
    
    def __getitem__(self, idx):
        if self.use_mosaic:
            image, target = self.mosaic(idx)
        else:
            image, target = self.base_dataset[idx]
        
        # Apply random augmentations
        if isinstance(image, torch.Tensor):
            from torchvision.transforms import functional as F
            image = F.to_pil_image(image)
        
        image, target = apply_random_augmentations(image, target)
        
        # Convert to tensor
        from torchvision.transforms import ToTensor
        image = ToTensor()(image)
        
        return image, target

# Create augmented training dataset
augmented_train_dataset = AugmentedDataset(
    train_dataset,
    use_mosaic=USE_MOSAIC_AUGMENTATION,
    mosaic_prob=MOSAIC_PROB
)

print(f"Created augmented training dataset with {len(augmented_train_dataset)} samples")

In [ ]:
# Visualize mosaic augmentation examples
if USE_MOSAIC_AUGMENTATION:
    print("Examples of Mosaic Augmentation:\n")
    display_sample_batch(
        augmented_train_dataset,
        num_samples=4,
        class_names=train_dataset.categories
    )

## 7. Create Data Loaders

In [ ]:
# Create data loaders
train_loader = data.DataLoader(
    augmented_train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=shuffle,
    sampler=train_sampler,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

val_loader = data.DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

## 8. Initialize Model

In [ ]:
# Create model
model = get_fasterrcnn_model(
    num_classes=train_dataset.num_classes,
    pretrained=True,
    trainable_backbone_layers=3
)

model.to(DEVICE)

# Setup optimizer and scheduler
optimizer = get_optimizer(model, lr=LEARNING_RATE)
lr_scheduler = get_lr_scheduler(optimizer, step_size=3, gamma=0.1)

print(f"Model initialized on {DEVICE}")
print(f"Optimizer: {optimizer.__class__.__name__}")
print(f"Initial learning rate: {optimizer.param_groups[0]['lr']}")

## 9. Training Loop with Augmentation

In [ ]:
# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'learning_rate': []
}

best_val_loss = float('inf')
CHECKPOINT_DIR = "checkpoints_augmented"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("Starting training with augmentation...\n")

In [ ]:
for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}")
    print(f"{'='*60}")
    
    # Train
    train_metrics = train_one_epoch(
        model, optimizer, train_loader, DEVICE, epoch + 1
    )
    
    # Evaluate
    val_metrics = evaluate(model, val_loader, DEVICE)
    
    # Update learning rate
    lr_scheduler.step()
    
    # Record history
    history['train_loss'].append(train_metrics['loss'])
    history['val_loss'].append(val_metrics['val_loss'])
    history['learning_rate'].append(optimizer.param_groups[0]['lr'])
    
    # Print summary
    print(f"\nEpoch {epoch + 1} Summary:")
    print(f"  Train Loss: {train_metrics['loss']:.4f}")
    print(f"  Val Loss: {val_metrics['val_loss']:.4f}")
    print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print(f"  Time: {train_metrics['time']:.2f}s")
    
    # Save best model
    if val_metrics['val_loss'] < best_val_loss:
        best_val_loss = val_metrics['val_loss']
        checkpoint_path = os.path.join(CHECKPOINT_DIR, "best_model_augmented.pth")
        save_model_checkpoint(
            model, optimizer, epoch + 1, val_metrics['val_loss'],
            checkpoint_path,
            additional_info={
                'train_loss': train_metrics['loss'],
                'use_class_aware_sampling': USE_CLASS_AWARE_SAMPLING,
                'use_mosaic_augmentation': USE_MOSAIC_AUGMENTATION
            }
        )
        print(f"  ✓ New best model saved!")
    
    # Save latest checkpoint
    latest_path = os.path.join(CHECKPOINT_DIR, "latest_model_augmented.pth")
    save_model_checkpoint(
        model, optimizer, epoch + 1, val_metrics['val_loss'],
        latest_path
    )

print("\n" + "="*60)
print("Training completed!")
print("="*60)

## 10. Plot Training History

In [ ]:
plot_training_history({
    'Training Loss': history['train_loss'],
    'Validation Loss': history['val_loss'],
    'Learning Rate': history['learning_rate']
})

## 11. Final Evaluation with mAP

Note: Using test set if available, otherwise validation set for final evaluation.

In [ ]:
# Prepare evaluation dataset and loader
# Use test set if available, otherwise use validation set
eval_dataset = test_dataset if has_test_set else val_dataset
eval_loader = data.DataLoader(
    eval_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=collate_fn
)

print(f"Evaluating on: {'Test' if has_test_set else 'Validation'} set")
print(f"Evaluation dataset size: {len(eval_dataset)}")

# Load best model for evaluation
best_checkpoint = torch.load(os.path.join(CHECKPOINT_DIR, "best_model_augmented.pth"))
model.load_state_dict(best_checkpoint['model_state_dict'])
model.to(DEVICE)

print(f"Loaded best model from epoch {best_checkpoint['epoch']}")

In [ ]:
# Evaluate on evaluation set (test or validation) at multiple IoU thresholds
print(f"Evaluating augmented model with mAP metric on {'test' if has_test_set else 'validation'} set...\n")

# Evaluate at multiple IoU thresholds
iou_thresholds = [0.5, 0.75, 0.9, 1.0]
all_map_results_augmented = {}

print("Evaluating at multiple IoU thresholds:")
print("=" * 50)

for iou_thresh in iou_thresholds:
    print(f"\nEvaluating at IoU threshold: {iou_thresh}")
    map_results = evaluate_map(
        model, eval_loader, DEVICE,
        iou_threshold=iou_thresh,
        confidence_threshold=0.05
    )
    all_map_results_augmented[f"mAP@{iou_thresh}"] = map_results['mAP']
    print(f"mAP@{iou_thresh}: {map_results['mAP']:.4f}")

# Store detailed results for IoU=0.5 (standard metric)
detailed_results_augmented = evaluate_map(
    model, eval_loader, DEVICE,
    iou_threshold=0.5,
    confidence_threshold=0.05
)

print("\n" + "=" * 50)
print("mAP Summary at Different IoU Thresholds:")
print("=" * 50)
for threshold, mAP_value in all_map_results_augmented.items():
    print(f"{threshold}: {mAP_value:.4f}")
print("=" * 50)

print("\nDetailed Results at IoU=0.5:")
print_metrics(detailed_results_augmented, class_names=eval_dataset.categories)

## 12. Compare with Baseline

In [ ]:
# Load baseline results if available
baseline_results_file = "../baseline_results.json"

if os.path.exists(baseline_results_file):
    with open(baseline_results_file, 'r') as f:
        baseline_results = json.load(f)
    
    print("Comparison with Baseline:\n")
    print(f"Baseline mAP@0.5: {baseline_results['mAP@0.5']:.4f}")
    print(f"Augmented mAP@0.5: {detailed_results_augmented['mAP']:.4f}")
    print(f"Improvement: {(detailed_results_augmented['mAP'] - baseline_results['mAP@0.5']):.4f}")
    print(f"Relative improvement: {((detailed_results_augmented['mAP'] / baseline_results['mAP@0.5']) - 1) * 100:.2f}%")
    
    print("\n" + "=" * 50)
    print("mAP Comparison at Different IoU Thresholds:")
    print("=" * 50)
    for threshold in ['mAP@0.5', 'mAP@0.75', 'mAP@0.9', 'mAP@1.0']:
        baseline_val = baseline_results.get('mAP_at_iou_thresholds', {}).get(threshold, 'N/A')
        augmented_val = all_map_results_augmented.get(threshold, 'N/A')
        if baseline_val != 'N/A' and augmented_val != 'N/A':
            print(f"{threshold}: Baseline={baseline_val:.4f}, Augmented={augmented_val:.4f}, Diff={augmented_val-baseline_val:.4f}")
        else:
            print(f"{threshold}: Baseline={baseline_val}, Augmented={augmented_val}")
    print("=" * 50)
    
    # Extract per-class AP for comparison (at IoU=0.5)
    baseline_ap = {int(k.split('_')[-1]): v for k, v in baseline_results['per_class_ap'].items()}
    augmented_ap = {int(k.split('_')[-1]): v for k, v in detailed_results_augmented.items() if k.startswith('AP_class_')}
    
    # Plot comparison
    plot_ap_comparison(
        baseline_ap,
        augmented_ap,
        class_names=val_dataset.categories
    )
else:
    print("Baseline results not found. Run the baseline notebook first.")

## 13. Visualize Predictions

In [ ]:
# Visualize predictions on random evaluation samples
model.eval()

num_visualizations = 6
indices = np.random.choice(len(eval_dataset), num_visualizations, replace=False)

print(f"Visualizing predictions from {'test' if has_test_set else 'validation'} set:\n")

for idx in indices:
    image, target = eval_dataset[idx]
    
    # Get prediction
    with torch.no_grad():
        prediction = model([image.to(DEVICE)])[0]
    
    # Visualize
    visualize_predictions(
        image,
        prediction,
        class_names=eval_dataset.categories,
        confidence_threshold=0.5
    )

## 14. Save Results

In [ ]:
# Save evaluation results
results_file = "augmented_results.json"

results = {
    'model': 'Faster R-CNN ResNet50-FPN (with augmentation)',
    'evaluated_on': 'test' if has_test_set else 'validation',
    'augmentation_techniques': {
        'class_aware_sampling': USE_CLASS_AWARE_SAMPLING,
        'mosaic_augmentation': USE_MOSAIC_AUGMENTATION,
        'mosaic_prob': MOSAIC_PROB
    },
    'num_epochs': NUM_EPOCHS,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'best_val_loss': float(best_val_loss),
    'final_train_loss': float(history['train_loss'][-1]),
    'mAP@0.5': float(detailed_results_augmented['mAP']),
    'mAP_at_iou_thresholds': {k: float(v) for k, v in all_map_results_augmented.items()},
    'per_class_ap': {k: float(v) for k, v in detailed_results_augmented.items() if k.startswith('AP_class_')},
    'training_history': history
}

with open(results_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved to {results_file}")
print(f"Note: Model was evaluated on the {'test' if has_test_set else 'validation'} set")
print(f"\nmAP Summary:")
for threshold, mAP_value in all_map_results_augmented.items():
    print(f"  {threshold}: {mAP_value:.4f}")

## Summary

This notebook demonstrated:

### Augmentation Techniques
1. **Class-Aware Sampling**: Balanced training by sampling images based on class distribution
   - Ensures minority classes are seen more frequently
   - Prevents model bias towards majority classes

2. **Mosaic Augmentation**: Combined 4 images into 2×2 grids
   - Increases diversity of object scales and contexts
   - Improves model robustness to different arrangements

### Expected Improvements
- Better performance on minority classes
- More robust detection across varied contexts
- Improved overall mAP

### Next Steps
1. Fine-tune augmentation parameters (mosaic probability, sampling strategy)
2. Experiment with additional augmentations (rotation, scaling, color jitter)
3. Try different model architectures (MobileNet for speed)
4. Implement the full Yahtzee web application